In [1]:
import numpy as np
import pandas as pd
import pickle
from matplotlib import pyplot as plt
from sklearn.ensemble import RandomForestRegressor
from scipy import stats

# Open model

In [2]:
with open("rf_model.pkl","rb") as f:
    rf_model = pickle.load(f)

# Load inference data set

In [3]:
X = np.loadtxt("inference.txt")
print(X.shape)

(838151, 9)


# Run inference

In [4]:
%%time
Y = rf_model.predict(X) # Units: m
np.savetxt("Inference_Y.txt", Y)

CPU times: user 7.03 s, sys: 1.32 s, total: 8.35 s
Wall time: 1.72 s
